In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import json
import zipfile
from sentence_transformers import CrossEncoder, SentenceTransformer

# Enable Mac M4 Acceleration (MPS)
if torch.backends.mps.is_available():
    device = 'mps'
    print("Apple Silicon M4 Detected: Using 'mps' acceleration.")
else:
    device = 'cpu'
    print("MPS not found. Running on CPU.")

In [ ]:
# CHANGE THIS:
input_a = 'test_track_a.jsonl'  # Was 'dev_track_a.jsonl'

# AND THIS:
input_b = 'test_track_b.jsonl'  # Was 'dev_track_b.jsonl'



In [4]:
# Check column names
print("Columns found:", df_a.columns.tolist())

# Show the first row to see the structure
pd.set_option('display.max_columns', None)
print(df_a.iloc[0])

Columns found: ['anchor_text', 'text_a', 'text_b', 'text_a_is_closer']
anchor_text         The book follows an international organization...
text_a              The old grandmother Tina arrives in town to at...
text_b              The nano-plague that poisoned Earth's water su...
text_a_is_closer                                                False
Name: 0, dtype: object


In [8]:
import json
import os
import zipfile
import glob
import pandas as pd

# --- PART 1: DIAGNOSE TRACK A SIZE MISMATCH ---
print("🔍 Checking for missing files...")
# List all JSONL files and their lengths to find the missing 200 items
jsonl_files = glob.glob("*.jsonl") + glob.glob("data/*.jsonl") + glob.glob("SemEval_Task4/*.jsonl")
total_rows = 0

for f in jsonl_files:
    try:
        count = sum(1 for line in open(f))
        print(f"   - Found file: {f} ({count} lines)")
    except Exception as e:
        print(f"   - Could not read {f}")

print("\n⚠️ CRITICAL: The server expects 400 items for Track A.")
print(f"   We currently have {len(df_a)} items in memory.")

# Check if we should combine files (Example logic)
# If you see another file in the list above (like 'train_track_a.jsonl'), 
# we might need to load that too. For now, we proceed with the Fix for Track B.

# --- PART 2: FIX TRACK B KEY NAME ---
print("\n🛠️ Applying Fix for Track B (Renaming 'embeddings' to 'embedding')...")

# Save Track B with the CORRECT key "embedding" (Singular)
with open('outputs/track_b.jsonl', 'w') as f:
    for emb in embeddings_list:
        # FIX: Changed "embeddings" -> "embedding"
        json.dump({"embedding": emb}, f)
        f.write('\n')

# Save Track A (No changes to logic, just saving current results)
with open('outputs/track_a.jsonl', 'w') as f:
    for val in df_a['predicted_a_is_closer']:
        json.dump({"text_a_is_closer": bool(val)}, f)
        f.write('\n')

# --- PART 3: VERIFY & ZIP ---
print("\n🔍 Final Verification:")
with open('outputs/track_b.jsonl', 'r') as f:
    first_line = json.loads(f.readline())
    if "embedding" in first_line:
        print("   ✅ Track B Key is correct: 'embedding'")
    else:
        print(f"   ❌ Track B Key is WRONG: {first_line.keys()}")

# Zip
zip_filename = 'submission_fixed.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('outputs/track_a.jsonl', arcname='track_a.jsonl')
    zipf.write('outputs/track_b.jsonl', arcname='track_b.jsonl')

print(f"\n🎉 FIXED SUBMISSION READY: '{zip_filename}'")

🔍 Checking for missing files...
   - Found file: dev_track_a.jsonl (200 lines)
   - Found file: dev_track_b.jsonl (479 lines)

⚠️ CRITICAL: The server expects 400 items for Track A.
   We currently have 200 items in memory.

🛠️ Applying Fix for Track B (Renaming 'embeddings' to 'embedding')...

🔍 Final Verification:
   ✅ Track B Key is correct: 'embedding'

🎉 FIXED SUBMISSION READY: 'submission_fixed.zip'


In [9]:
input_a = 'test_track_a.jsonl'
df_a = pd.read_json(input_a, lines=True)
print(f"Loaded {len(df_a)} pairs for Track A.")

# We use the Cross-Encoder to compare the stories
model_a = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

print("Running Track A")

# We pair the anchor with text_a, and anchor with text_b
pairs_a = df_a[['anchor_text', 'text_a']].values.tolist()
pairs_b = df_a[['anchor_text', 'text_b']].values.tolist()

print("Scoring pairs...")
scores_a = model_a.predict(pairs_a, batch_size=32, show_progress_bar=True)
scores_b = model_a.predict(pairs_b, batch_size=32, show_progress_bar=True)

df_a['predicted_a_is_closer'] = scores_a > scores_b

print("Track A Done.")

Loaded 400 pairs for Track A.
Running Track A
Scoring pairs...


Batches: 100%|██████████████████████████████████| 13/13 [00:04<00:00,  3.08it/s]

Track A Done.


In [6]:
input_b = 'dev_track_b.jsonl'
df_b = pd.read_json(input_b, lines=True)
print(f"Loaded {len(df_b)} stories for Track B.")

# Added 'anchor_text' to the search list
possible_cols = ['anchor_text', 'anchor', 'text', 'story']
col_name = next((c for c in possible_cols if c in df_b.columns), None)

if col_name:
    print(f"Encoding column: '{col_name}'")
    
    model_b = SentenceTransformer('all-mpnet-base-v2', device=device)
    print("Running Track B on M4...")

    embeddings = model_b.encode(
        df_b[col_name].tolist(), 
        batch_size=32, 
        show_progress_bar=True, 
        device=device,
        convert_to_numpy=True
    )
    
    embeddings_list = embeddings.tolist()
    print(f"Track B Done. Generated {len(embeddings_list)} vectors.")

else:
    print(f"ERROR: Columns found were {df_b.columns.tolist()}. Please update the code to match.")

Loaded 479 stories for Track B.
Encoding column: 'text'
Running Track B on M4...


Batches: 100%|██████████████████████████████████| 15/15 [00:12<00:00,  1.19it/s]

Track B Done. Generated 479 vectors.


In [7]:
import json
import os
import zipfile

os.makedirs('outputs', exist_ok=True)

print("Saving Track A...")
with open('outputs/track_a.jsonl', 'w') as f:
    for val in df_a['predicted_a_is_closer']:
        json.dump({"text_a_is_closer": bool(val)}, f)
        f.write('\n')

print("Saving Track B...")
with open('outputs/track_b.jsonl', 'w') as f:
    for emb in embeddings_list:
        json.dump({"embeddings": emb}, f)
        f.write('\n')

print("\nVerifying files...")
try:
    with open('outputs/track_a.jsonl', 'r') as f:
        print(f"   - Track A check: {json.loads(f.readline())}") 
    with open('outputs/track_b.jsonl', 'r') as f:
        data = json.loads(f.readline())
        print(f"   - Track B check: Vector length is {len(data['embeddings'])}") # Should be 768
except Exception as e:
    print(f"Error during verification: {e}")

# 3. Zip for Submission
zip_filename = 'submission.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('outputs/track_a.jsonl', arcname='track_a.jsonl')
    zipf.write('outputs/track_b.jsonl', arcname='track_b.jsonl')

print(f"Finished.")

Saving Track A...
Saving Track B...

🔍 Verifying files...
   - Track A check: {'text_a_is_closer': False}
   - Track B check: Vector length is 768
Finished.


In [10]:
import pandas as pd
import glob
import os
import json
import zipfile
from sentence_transformers import CrossEncoder, SentenceTransformer
import torch

# --- 1. FIND THE CORRECT FILES BY COUNT ---
print("🔍 Scanning for Test Data...")
jsonl_files = glob.glob("*.jsonl") + glob.glob("data/*.jsonl") + glob.glob("SemEval_Task4/*.jsonl")

input_a = None
input_b = None

for f in jsonl_files:
    try:
        # Count lines in file
        count = sum(1 for line in open(f))
        
        # Track A Test File must have exactly 400 lines
        if count == 400:
            input_a = f
            print(f"   ✅ Found Track A Test File: {f} (400 items)")
            
        # Track B Test File usually has around 849 lines (based on your info)
        elif count == 849:
            input_b = f
            print(f"   ✅ Found Track B Test File: {f} (849 items)")
            
    except:
        pass

if not input_a or not input_b:
    print("⚠️ WARNING: Could not auto-detect files. Please ensure you unzipped the Test Data!")
    # Fallback to manual filenames if auto-detection fails
    if not input_a: input_a = 'test_track_a.jsonl' 
    if not input_b: input_b = 'test_track_b.jsonl'

# --- 2. SETUP AI MODELS ---
if torch.backends.mps.is_available():
    device = 'mps'
    print("🚀 Using Apple M4 (MPS)")
else:
    device = 'cpu'
    print("⚠️ Using CPU")

model_a = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)
model_b = SentenceTransformer('all-mpnet-base-v2', device=device)

# --- 3. PROCESS TRACK A (400 items) ---
print(f"\n🧠 Processing Track A ({input_a})...")
df_a = pd.read_json(input_a, lines=True)

# Detect column names for Track A
anc_col = next((c for c in ['anchor_text', 'anchor'] if c in df_a.columns), 'anchor')
a_col = next((c for c in ['text_a', 'a'] if c in df_a.columns), 'a')
b_col = next((c for c in ['text_b', 'b'] if c in df_a.columns), 'b')

# Create pairs
pairs_a = df_a[[anc_col, a_col]].values.tolist()
pairs_b = df_a[[anc_col, b_col]].values.tolist()

# Predict
scores_a = model_a.predict(pairs_a, batch_size=32, show_progress_bar=True)
scores_b = model_a.predict(pairs_b, batch_size=32, show_progress_bar=True)
preds_a = scores_a > scores_b

# --- 4. PROCESS TRACK B (849 items) ---
print(f"\n🧠 Processing Track B ({input_b})...")
df_b = pd.read_json(input_b, lines=True)

# Detect column for Track B (Input is usually "text" or "story")
text_col = next((c for c in ['text', 'story', 'anchor', 'anchor_text'] if c in df_b.columns), None)

if text_col:
    embeddings = model_b.encode(
        df_b[text_col].tolist(), 
        batch_size=32, 
        show_progress_bar=True, 
        device=device,
        convert_to_numpy=True
    )
    embeddings_list = embeddings.tolist()
else:
    print("❌ ERROR: Could not find text column for Track B")
    embeddings_list = []

# --- 5. SAVE & ZIP (Using 'embedding' singular) ---
os.makedirs('outputs', exist_ok=True)

# Save Track A
with open('outputs/track_a.jsonl', 'w') as f:
    for val in preds_a:
        json.dump({"text_a_is_closer": bool(val)}, f)
        f.write('\n')

# Save Track B
with open('outputs/track_b.jsonl', 'w') as f:
    for emb in embeddings_list:
        # IMPORTANT: Using 'embedding' (singular) as per error log
        json.dump({"embedding": emb}, f)
        f.write('\n')

# Zip
zip_name = 'submission_test_final.zip'
with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('outputs/track_a.jsonl', arcname='track_a.jsonl')
    zipf.write('outputs/track_b.jsonl', arcname='track_b.jsonl')

print(f"\n🎉 DONE! Upload '{zip_name}' to CodaBench Testing Phase.")

🔍 Scanning for Test Data...
   ✅ Found Track B Test File: test_track_b.jsonl (849 items)
   ✅ Found Track A Test File: test_track_a.jsonl (400 items)
🚀 Using Apple M4 (MPS)

🧠 Processing Track A (test_track_a.jsonl)...


Batches: 100%|██████████████████████████████████| 13/13 [00:03<00:00,  3.49it/s]



🧠 Processing Track B (test_track_b.jsonl)...


Batches: 100%|██████████████████████████████████| 27/27 [00:16<00:00,  1.61it/s]



🎉 DONE! Upload 'submission_test_final.zip' to CodaBench Testing Phase.


In [ ]:
import pandas as pd
import glob
import os
import json
import zipfile
from sentence_transformers import CrossEncoder, SentenceTransformer
import torch

# --- 1. HARDWARE SETUP ---
# We force MPS (Metal Performance Shaders) for Mac M4
if torch.backends.mps.is_available():
    device = 'mps'
    print("🚀 POWER UNLEASHED: Using Apple M4 (MPS) Acceleration")
else:
    device = 'cpu'
    print("⚠️ WARNING: Running on CPU. This will be very slow.")

# --- 2. INTELLIGENT FILE FINDER ---
print("\n🔍 Scanning for the correct Test Data...")
# We look for files matching the specific line counts of the Test Phase
jsonl_files = glob.glob("*.jsonl") + glob.glob("data/*.jsonl") + glob.glob("SemEval_Task4/*.jsonl")

input_a = None
input_b = None

for f in jsonl_files:
    try:
        count = sum(1 for line in open(f))
        if count == 400:
            input_a = f
            print(f"   ✅ LOCK: Track A Test File found: {f} (400 items)")
        elif count == 849:
            input_b = f
            print(f"   ✅ LOCK: Track B Test File found: {f} (849 items)")
    except: pass

if not input_a or not input_b:
    print("❌ CRITICAL ERROR: Could not find files with 400 or 849 lines.")
    print("Please download the 'Evaluation Data' from the website again.")
    # Fallback to manual names just in case
    input_a = 'test_track_a.jsonl'
    input_b = 'test_track_b.jsonl'

# --- 3. LOAD THE "GOD TIER" MODELS ---
print("\n🔽 Loading SOTA Models (This involves large downloads, please wait)...")

# Track A: DeBERTa v3 Large (The best Open Source pairwise classifier)
model_a_name = 'cross-encoder/nli-deberta-v3-large'
print(f"   - Loading Track A Model: {model_a_name}...")
model_a = CrossEncoder(model_a_name, device=device)

# Track B: GTE Large v1.5 (The best Open Source embedding model)
# trust_remote_code=True is required for this specific high-end model
model_b_name = 'Alibaba-NLP/gte-large-en-v1.5'
print(f"   - Loading Track B Model: {model_b_name}...")
model_b = SentenceTransformer(model_b_name, trust_remote_code=True, device=device)


# --- 4. EXECUTE TRACK A (HIGH PRECISION) ---
print(f"\n🧠 ANALYZING TRACK A ({input_a})...")
df_a = pd.read_json(input_a, lines=True)

# Smart column detection
anc_col = next((c for c in ['anchor_text', 'anchor'] if c in df_a.columns), 'anchor')
a_col = next((c for c in ['text_a', 'a'] if c in df_a.columns), 'a')
b_col = next((c for c in ['text_b', 'b'] if c in df_a.columns), 'b')

# Create pairs
pairs_a = df_a[[anc_col, a_col]].values.tolist()
pairs_b = df_a[[anc_col, b_col]].values.tolist()

# INFERENCE
# Batch size 4 is critical for 'Large' models on Mac to avoid crashing
print("   - Scoring pairs (Slow & Steady)...")
scores_a = model_a.predict(pairs_a, batch_size=4, show_progress_bar=True)
scores_b = model_a.predict(pairs_b, batch_size=4, show_progress_bar=True)

# Logic: Higher score = closer entailment/similarity
preds_a = scores_a > scores_b


# --- 5. EXECUTE TRACK B (DEEP SEMANTICS) ---
print(f"\n🧠 EMBEDDING TRACK B ({input_b})...")
df_b = pd.read_json(input_b, lines=True)

# Smart column detection
text_col = next((c for c in ['text', 'story', 'anchor', 'anchor_text'] if c in df_b.columns), None)

if text_col:
    print(f"   - Encoding column: '{text_col}'")
    # INFERENCE
    # Batch size 4 ensures we fit the massive 8192 token window in memory
    embeddings = model_b.encode(
        df_b[text_col].tolist(), 
        batch_size=4, 
        show_progress_bar=True, 
        device=device,
        convert_to_numpy=True
    )
    embeddings_list = embeddings.tolist()
else:
    print("❌ ERROR: No text column found for Track B")
    embeddings_list = []


# --- 6. PACKAGING ---
print("\n📦 Packaging Submission...")
os.makedirs('outputs', exist_ok=True)

# Save Track A
with open('outputs/track_a.jsonl', 'w') as f:
    for val in preds_a:
        json.dump({"text_a_is_closer": bool(val)}, f)
        f.write('\n')

# Save Track B (Using singular 'embedding' key)
with open('outputs/track_b.jsonl', 'w') as f:
    for emb in embeddings_list:
        json.dump({"embedding": emb}, f)
        f.write('\n')

# Zip
zip_name = 'submission_SOTA_FINAL.zip'
with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('outputs/track_a.jsonl', arcname='track_a.jsonl')
    zipf.write('outputs/track_b.jsonl', arcname='track_b.jsonl')

print(f"\n🏆 VICTORY: '{zip_name}' is ready.")
print("   Upload this to CodaBench Testing Phase.")

🚀 POWER UNLEASHED: Using Apple M4 (MPS) Acceleration

🔍 Scanning for the correct Test Data...
   ✅ LOCK: Track B Test File found: test_track_b.jsonl (849 items)
   ✅ LOCK: Track A Test File found: test_track_a.jsonl (400 items)

🔽 Loading SOTA Models (This involves large downloads, please wait)...
   - Loading Track A Model: cross-encoder/nli-deberta-v3-large...


In [ ]:
import pandas as pd
import glob
import os
import json
import zipfile
from sentence_transformers import CrossEncoder, SentenceTransformer
from huggingface_hub import snapshot_download # This helps us show download bars
import torch

# --- 1. HARDWARE SETUP ---
if torch.backends.mps.is_available():
    device = 'mps'
    print("🚀 POWER UNLEASHED: Using Apple M4 (MPS) Acceleration")
else:
    device = 'cpu'
    print("⚠️ WARNING: Running on CPU.")

# --- 2. FIND FILES ---
print("\n🔍 Scanning for Test Data...")
jsonl_files = glob.glob("*.jsonl") + glob.glob("data/*.jsonl") + glob.glob("SemEval_Task4/*.jsonl")
input_a, input_b = None, None

for f in jsonl_files:
    try:
        count = sum(1 for line in open(f))
        if count == 400: input_a = f
        elif count == 849: input_b = f
    except: pass

if not input_a or not input_b:
    input_a, input_b = 'test_track_a.jsonl', 'test_track_b.jsonl'
    print("⚠️ Using manual filenames (Auto-detection failed)")
else:
    print(f"   ✅ Track A File: {input_a}")
    print(f"   ✅ Track B File: {input_b}")

# --- 3. DOWNLOAD MODELS WITH PROGRESS BARS ---
print("\n⬇️ STARTING DOWNLOADS (This ensures you see progress)...")

# Model 1: DeBERTa v3 Large
print("   1. Downloading DeBERTa-v3-Large (~800MB)...")
model_a_id = 'cross-encoder/nli-deberta-v3-large'
snapshot_download(repo_id=model_a_id) # This triggers the bar

# Model 2: GTE Large
print("   2. Downloading GTE-Large-v1.5 (~1.5GB)...")
model_b_id = 'Alibaba-NLP/gte-large-en-v1.5'
snapshot_download(repo_id=model_b_id) # This triggers the bar

print("\n✅ Downloads Complete. Loading into Memory...")
model_a = CrossEncoder(model_a_id, device=device)
model_b = SentenceTransformer(model_b_id, trust_remote_code=True, device=device)

# --- 4. EXECUTE TRACK A ---
print(f"\n🧠 SCORING TRACK A ({input_a})...")
df_a = pd.read_json(input_a, lines=True)

# Column detection
anc_col = next((c for c in ['anchor_text', 'anchor'] if c in df_a.columns), 'anchor')
a_col = next((c for c in ['text_a', 'a'] if c in df_a.columns), 'a')
b_col = next((c for c in ['text_b', 'b'] if c in df_a.columns), 'b')

# Create pairs
pairs_a = df_a[[anc_col, a_col]].values.tolist()
pairs_b = df_a[[anc_col, b_col]].values.tolist()

# INFERENCE (Progress bar included)
scores_a = model_a.predict(pairs_a, batch_size=4, show_progress_bar=True)
scores_b = model_a.predict(pairs_b, batch_size=4, show_progress_bar=True)
preds_a = scores_a > scores_b

# --- 5. EXECUTE TRACK B ---
print(f"\n🧠 EMBEDDING TRACK B ({input_b})...")
df_b = pd.read_json(input_b, lines=True)
text_col = next((c for c in ['text', 'story', 'anchor', 'anchor_text'] if c in df_b.columns), None)

if text_col:
    # INFERENCE (Progress bar included)
    embeddings = model_b.encode(
        df_b[text_col].tolist(), 
        batch_size=4, 
        show_progress_bar=True, 
        device=device,
        convert_to_numpy=True
    )
    embeddings_list = embeddings.tolist()
else:
    embeddings_list = []

# --- 6. ZIP IT UP ---
print("\n📦 Zipping Final Submission...")
os.makedirs('outputs', exist_ok=True)

with open('outputs/track_a.jsonl', 'w') as f:
    for val in preds_a:
        json.dump({"text_a_is_closer": bool(val)}, f)
        f.write('\n')

with open('outputs/track_b.jsonl', 'w') as f:
    for emb in embeddings_list:
        json.dump({"embedding": emb}, f)
        f.write('\n')

zip_name = 'submission_SOTA_FINAL.zip'
with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('outputs/track_a.jsonl', arcname='track_a.jsonl')
    zipf.write('outputs/track_b.jsonl', arcname='track_b.jsonl')

print(f"\n🏆 READY! Upload '{zip_name}' to CodaBench Testing Phase.")